<a href="https://colab.research.google.com/github/Jaidon-Smith/AI-Karaoke/blob/main/AI_Forced_Alignment_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# intel-tensorflow is required to run on CPU
!pip install torch torchaudio soundfile omegaconf intel-tensorflow tensorflow_hub

     |████████████████████████████████| 7.6MB 7.4MB/s 
     |████████████████████████████████| 165.8MB 36kB/s 
     |████████████████████████████████| 645kB 27.0MB/s 
ERROR: torchaudio 0.7.2 has requirement torch==1.7.1, but you'll have torch 1.7.0+cu101 which is incompatible.
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
#speech_file = 'speech_orig.wav'
speech_file = '/content/recorded.wav'


import os
import torch
import tensorflow as tf
import tensorflow_hub as tf_hub
from omegaconf import OmegaConf

language = 'en'

# load provided utils using torch.hub for brevity
_, decoder, utils = torch.hub.load('snakers4/silero-models', model='silero_stt', language=language)
(read_batch, split_into_batches,
 read_audio, prepare_model_input) = utils

# see available models
torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml', 'models.yml')
models = OmegaConf.load('models.yml')
available_languages = list(models.stt_models.keys())
assert language in available_languages

# load the actual tf model
tf_model = tf_hub.load(models.stt_models.get(language).latest.tf)

# download a single file, any format compatible with TorchAudio (soundfile backend)
torch.hub.download_url_to_file(models.stt_models.get(language).latest.meta.sample, dst = speech_file, progress=True)
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# tf inference
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
print(decoder(torch.Tensor(res.numpy())[0]))


Downloading: "https://github.com/snakers4/silero-models/archive/master.zip" to /root/.cache/torch/hub/master.zip
/usr/local/lib/python3.6/dist-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '
/usr/local/lib/python3.6/dist-packages/torchaudio/backend/utils.py:64: UserWarning: The interface of "soundfile" backend is planned to change in 0.8.0 to match that of "sox_io" backend and the current interface will be removed in 0.9.0. To use the new interface, do `torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE = False` before setting the backend to "soundfile". Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  'The interface of "soundfile" backend is planned to change in 0.8.0 


the birch canoe slit on the smooth planks blared the sheet to the dark blue background it's easy to tell a depth of a well four hours of steady work faced us


In [3]:
speech_file = '/content/recorded.wav'
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# tf inference
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
print(decoder(torch.Tensor(res.numpy())[0]))

the birch canoe slit on the smooth planks blared the sheet to the dark blue background it's easy to tell a depth of a well four hours of steady work faced us


In [4]:
batches

[['/content/recorded.wav']]

In [5]:
tf_model

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7f34af0e1b70>

In [6]:
res

<tf.Tensor: shape=(1, 136, 999), dtype=float32, numpy=
array([[[9.9562067e-01, 2.1983162e-06, 4.2956495e-03, ...,
         7.4658083e-06, 6.9315170e-06, 1.2493901e-05],
        [2.3445139e-02, 1.3809478e-05, 9.7602081e-01, ...,
         3.1058391e-06, 1.8454934e-06, 3.6289461e-05],
        [2.6382526e-02, 6.0946086e-08, 8.4972737e-05, ...,
         1.2406174e-05, 1.3228331e-06, 9.7339547e-01],
        ...,
        [9.9961782e-01, 1.4285623e-06, 9.0514386e-06, ...,
         6.1206483e-05, 6.5136401e-06, 2.0248130e-05],
        [9.9961782e-01, 1.4285079e-06, 9.0514040e-06, ...,
         6.1204380e-05, 6.5133045e-06, 2.0246334e-05],
        [9.9961782e-01, 1.4285732e-06, 9.0529056e-06, ...,
         6.1196843e-05, 6.5128511e-06, 2.0243941e-05]]], dtype=float32)>

In [7]:
decoder

In [8]:
# construct a shape[1,136,999] tensor where the columns consist of a single 1 in one of the 136 and the others 0
# In this way I may find out what the 136 means

import numpy as np
value = 0
the_tensor = []
for i in range(136):
  if i == value:
    the_tensor.append(list(np.ones(999)))
  else:
    the_tensor.append(list(np.zeros(999)))

the_tensor = np.array(the_tensor)
the_tensor = the_tensor[np.newaxis, :]
#print(the_tensor.shape)
the_tensor = tf.convert_to_tensor(the_tensor)
print(the_tensor)


tf.Tensor(
[[[1. 1. 1. ... 1. 1. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]], shape=(1, 136, 999), dtype=float64)


In [9]:
#res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
print(decoder(torch.Tensor(the_tensor.numpy())[0]))

# Test that the 999 does refer to time, and thus halving it will shorten the output
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
print(decoder(torch.Tensor(res.numpy())[0]))


the birch canoe slit on the smooth planks blared the sheet to the dark blue background it's easy to tell a depth of a well four hours of steady work faced us


In [10]:
res1, res2 = tf.split(res, [500,499],2)
print(res1.shape)
#print(decoder(torch.Tensor(res1.numpy())[0]))
'''---------------------------------------------------------------------------
AssertionError                            Traceback (most recent call last)
<ipython-input-38-617167afc803> in <module>()
      1 res1, res2 = tf.split(res, [500,499],2)
      2 print(res1.shape)
----> 3 print(decoder(torch.Tensor(res1.numpy())[0]))

1 frames
/root/.cache/torch/hub/snakers4_silero-models_master/utils.py in process(self, probs, wav_len, word_align)
     62     def process(self,
     63                 probs, wav_len, word_align):
---> 64         assert len(self.labels) == probs.shape[1]
     65         for_string = []
     66         argm = torch.argmax(probs, axis=1)

AssertionError: '''

(1, 136, 500)


'---------------------------------------------------------------------------\nAssertionError                            Traceback (most recent call last)\n<ipython-input-38-617167afc803> in <module>()\n      1 res1, res2 = tf.split(res, [500,499],2)\n      2 print(res1.shape)\n----> 3 print(decoder(torch.Tensor(res1.numpy())[0]))\n\n1 frames\n/root/.cache/torch/hub/snakers4_silero-models_master/utils.py in process(self, probs, wav_len, word_align)\n     62     def process(self,\n     63                 probs, wav_len, word_align):\n---> 64         assert len(self.labels) == probs.shape[1]\n     65         for_string = []\n     66         argm = torch.argmax(probs, axis=1)\n\nAssertionError: '

In [11]:
# construct a shape[1,136,999] tensore where the columns consist of a single 1 in one of the 136 and the others 0
# In this way I may find out what the 136 means

import numpy as np
value = 11
the_tensor = []
for i in range(999):
  if i == value:
    temp = list(np.ones(136))
    temp[10] = 0
    the_tensor.append(temp)
  else:
    temp = list(np.zeros(136))
    the_tensor.append(temp)
    if i == 12:
      temp[10] = 1


the_tensor = np.array(the_tensor)
the_tensor = np.swapaxes(the_tensor, 0, 1)
the_tensor = the_tensor[np.newaxis, :]
#print(the_tensor.shape)
the_tensor = tf.convert_to_tensor(the_tensor)
print(the_tensor)

print(decoder(torch.Tensor(the_tensor.numpy())[0]))

tf.Tensor(
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]], shape=(1, 136, 999), dtype=float64)
entoen


In [12]:
#@title Install dependencies

!pip install -q omegaconf
!pip install -q torchaudio
!pip install -q soundfile
!pip install -q pydub

import os
from os.path import exists

if not exists('silero-models'):
  !git clone -q --depth 1 https://github.com/snakers4/silero-models

%cd silero-models

# silero imports
import torch
import random
from glob import glob
from omegaconf import OmegaConf
from utils import (init_jit_model, 
                   split_into_batches,
                   read_audio,
                   read_batch,
                   prepare_model_input)
from colab_utils import (record_audio,
                         audio_bytes_to_np,
                         upload_audio)

device = torch.device('cpu')   # you can use any pytorch device
models = OmegaConf.load('models.yml')

# imports for uploading/recording
import numpy as np
import ipywidgets as widgets
from scipy.io import wavfile
from IPython.display import Audio, display, clear_output
from torchaudio.functional import vad


     |████████████████████████████████| 776.8MB 21kB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.7.1 which is incompatible.
/content/silero-models


In [13]:
sample_rate = 16000

clear_output()
audio = record_audio(10)
wavfile.write('/content/recorded.wav', sample_rate, (32767*audio).numpy().astype(np.int16))


#button = widgets.Button(description="Record Speech")
#button.on_click(_record_audio)
#display(button)

display(Audio(audio, rate=sample_rate, autoplay=True))
wavfile.write('test.wav', sample_rate, (32767*audio).numpy().astype(np.int16))
  

Starting recording for 10 seconds...


<IPython.core.display.Javascript object>

Finished recording!


In [14]:
len(decoder.labels)

999

In [15]:
decoder.blank_idx

0

In [16]:
speech_file = '/content/recorded.wav'
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# For each frame print the grapheme that is the most likely
graphemes = decoder.labels
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
res_array = np.array(res)
res_array[0].shape
result = np.argmax(res_array, axis=2)

text = []
for i in result[0].tolist():
  text.append(graphemes[i])

"".join(text)

'______________________________ _the _sun________  _and_______   _her__     _sh______________________________________________he________'

In [17]:
result = np.argmax(res_array, axis=2)

#result = f(result)

In [18]:
result[0].tolist()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 998,
 0,
 2,
 998,
 0,
 887,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 998,
 998,
 0,
 13,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 998,
 998,
 998,
 0,
 109,
 0,
 0,
 998,
 998,
 998,
 998,
 998,
 0,
 143,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 23,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [21]:
#Testing singing
speech_file = '/content/vocals.wav'
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# For each frame print the grapheme that is the most likely
graphemes = decoder.labels
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
res_array = np.array(res)
res_array[0].shape
result = np.argmax(res_array, axis=2)

text = []
for i in result[0].tolist():
  text.append(graphemes[i])

"".join(text)

'___________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________what_  _not   _believe__       __eyes_____  if   can_ _million  _f__ire_   f_li________ _when       the all_   as i  fellll____ly_____________________________________because  _they  _fell      open____ ________    le_    your ddro_ps _every____where_   you   _  me_   with   but   i  would  _just  stand_  and________  _st_____________________________i_  _like    make_   my____self_______________________________________________________________________________________________________ _it  _hard_ to say_     i  _ratherther__________________  i_ ___________________ _because_  _every_______  is   _never_______  to _see_________________________________________i   _get   a _thousand___  ____________ from   and  thousand__ 

In [ ]:
decoder.blank_idx

In [ ]:
np.argsort(res_array, axis=2)[0][100]
#result = np.argmax(res_array, axis=2)
#result[0][100]

In [ ]:



#Testing singing with empty removed
speech_file = '/content/fireflies_voc_1min.wav'
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# For each frame print the grapheme that is the most likely
graphemes = decoder.labels
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
res_array = np.array(res)
res_array[0].shape
result = np.argmax(res_array, axis=2)

original = result[0].copy()
for i,val in enumerate(original):
  #print(val)
  if val == 0:
    #print(np.argsort(res_array, axis=2)[0][i][-2])
    #result[0][i] = np.argsort(res_array, axis=2)[0][i][-2]
    result[0][i] = np.argpartition(res_array, -2, axis=2)[0][i][-2]

text = []
for i in result[0].tolist():
  text.append(graphemes[i])

"".join(text)


In [ ]:



#Testing singing with empty removed
speech_file = '/content/jepsen_voc.wav'
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# For each frame print the grapheme that is the most likely
graphemes = decoder.labels
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
res_array = np.array(res)
res_array[0].shape
result = np.argmax(res_array, axis=2)

#original = result[0].copy()
#for i,val in enumerate(original):
  #print(val)
#  if val == 0:
    #print(np.argsort(res_array, axis=2)[0][i][-2])
    #result[0][i] = np.argsort(res_array, axis=2)[0][i][-2]
#    result[0][i] = np.argpartition(res_array, -2, axis=2)[0][i][-2]

text = []
for i in result[0].tolist():
  text.append(graphemes[i])

"".join(text)


In [ ]:
decoder.labels

In [23]:
import math

subproblems = []
def minimal_token(text, tokens):
  global subproblems
  subproblems = [None for i in range(len(text))]
  solution = minimal_token_recursive(text, 0, tokens)
  return solution

def minimal_token_recursive(text, index, tokens):
  global subproblems
  if index >= len(text):
    return (0,[])
  # If we haven't calculated the solution before, calculate it now
  if subproblems[index] is None:
    subproblems[index] = minimal_token_recursive_calculate(text, index, tokens)
    
  return subproblems[index]
  


def minimal_token_recursive_calculate(text, index, tokens):
  minimum = None
  minimum_token = None
  for i, token in enumerate(tokens):
    if str.find(text[index:], token) == 0:
      subproblem = minimal_token_recursive(text, index+len(token), tokens)
      if subproblem is None:
        raise Exception("Sub none")
      if minimum is None or subproblem[0] < minimum[0]:
        minimum = subproblem
        minimum_token = i
  if minimum == None:
    #raise Exception("This text cannot be represented")
    # or instead just return something with a large cost
    return (math.inf, [])
  
  new_list = minimum[1].copy()
  new_list.insert(0, minimum_token)
  return (1 + minimum[0], new_list)

  


In [ ]:
#minimal_token("the inovation sensation", decoder.labels)
minimal_token("the", decoder.labels)

In [ ]:
decoder.

In [ ]:
str.find("hello", "ll")

In [ ]:
"hello"[0:]

In [ ]:
k = [1,2]
k.insert(2,0)
k

In [ ]:
minimal_token("hello", ["he", "llo", "l", "lo", "o"])

In [ ]:
minimal_token("hello", ["he", "llo", "l", "lo"])

In [ ]:
test = [1].append(k)
test

In [ ]:
res.shape

In [ ]:
the_string = '''look at this stuff isnt it neat wouldn't you think my collection's complete wouldn't you think i'm the girl girl who has everything'''

minimal_token(the_string, decoder.labels)

In [24]:
import math


subproblems = []
def forced_alignment(correct_text, text_from_speech):
  global subproblems

  time_values = np.array(text_from_speech).shape[0]

  for i in range(len(correct_text)):
    subproblems.append([])
    for j in range(time_values):
      subproblems[-1].append(None)

  for i in range(len(correct_text)):
    for j in range(time_values):
      forced_alignment_recursive(correct_text, text_from_speech, i, j)

  
  solution = forced_alignment_recursive(correct_text, text_from_speech, len(correct_text) - 1, time_values - 1)
  return solution

def forced_alignment_recursive(correct_text, text_from_speech, text_index, time):
  global subproblems
  
  if time < text_index or text_index < 0:
    return (-math.inf, [])
  if time == 0:
    return (0, [0])
  

  # If we haven't calculated the solution before, calculate it now
  if subproblems[text_index][time] is None:
    subproblems[text_index][time] = forced_alignment_recursive_calculate(correct_text, text_from_speech, text_index, time)
    
  return subproblems[text_index][time]
  


def forced_alignment_recursive_calculate(correct_text, text_from_speech, text_index, time):
  text = correct_text[text_index]
  probability = math.log(text_from_speech[time][text])

  


  

  
  options = [forced_alignment_recursive(correct_text, text_from_speech, text_index, time - 1),
             forced_alignment_recursive(correct_text, text_from_speech, text_index - 1, time - 1),
  ]

  option_probabilities = [i[0] for i in options]
  options_index = np.argmax(option_probabilities)

  indexes = options[options_index][1].copy()
  indexes.append(text_index)
  return (options[options_index][0] + probability, indexes)







  '''minimum = None
  minimum_token = None
  for i, token in enumerate(tokens):
    if str.find(text[index:], token) == 0:
      subproblem = forced_alignment_recursive(text, index+len(token), tokens)
      if subproblem is None:
        raise Exception("Sub none")
      if minimum is None or subproblem[0] < minimum[0]:
        minimum = subproblem
        minimum_token = i
  if minimum == None:
    #raise Exception("This text cannot be represented")
    # or instead just return something with a large cost
    return (math.inf, [])
  
  new_list = minimum[1].copy()
  new_list.insert(0, minimum_token)
  return (1 + minimum[0], new_list)'''

  


In [ ]:
test = [1,3]
np.argmax(test)

In [ ]:
math.log(5)

In [32]:
import string
text = '''you would not believe your eyes if ten million fireflies lit up the world as i fell asleep cause they fill the open air and leave teardrops everywhere you'd think me rude but i would just stand and stare i'd like to make myself believe that planet earth turns slowly it's hard to say that i'd rather stay awake when i'm asleep cause everything is never as it seems cause i'd get a thousand hugs from ten thousand lightning bugs as they tried to teach me how to dance a foxtrot above my head'''

text = '''You would not believe your eyes
If ten million fireflies
Lit up the world as I fell asleep
'Cause they fill the open air
And leave teardrops everywhere
You'd think me rude but I would just stand and stare
I'd like to make myself believe that planet earth turns slowly
It's hard to say that I'd rather stay awake when I'm asleep
'Cause everything is never as it seems
'Cause I'd get a thousand hugs
From ten thousand lightning bugs
As they tried to teach me how to dance
A foxtrot above my head
A sock hop beneath my bed
A disco ball is just hanging by a thread (thread, thread)
I'd like to make myself believe that planet earth turns slowly
It's hard to say that I'd rather stay awake when I'm asleep
'Cause everything is never as it seems (when I fall asleep)
Leave my door open just a crack
Please take me away from here
'Cause I feel like such an insomniac
Please take me away from here
Why do I tire of counting sheep?
Please take me away from here
When I'm far too tired to fall asleep
To ten million fireflies
I'm weird 'cause I hate goodbyes
I got misty eyes as they said farewell (they said farewell)
But I'll know where several are
If my dreams get real bizarre
'Cause I saved a few and I keep them in a jar (jar, jar, jar)
I'd like to make myself believe that planet earth turns slowly
It's hard to say that I'd rather stay awake when I'm asleep
'Cause everything is never as it seems (when I fall asleep)
I'd like to make myself believe that planet earth turns slowly
It's hard to say that I'd rather stay awake when I'm asleep
'Cause everything is never as it seems (when I fall asleep)'''

def parse_lyrics(text):
  new = text.lower()
  return new

text = parse_lyrics(text)
print(text)
  #Remove capitals
string.punctuation
string.ascii_letters

you would not believe your eyes
if ten million fireflies
lit up the world as i fell asleep
'cause they fill the open air
and leave teardrops everywhere
you'd think me rude but i would just stand and stare
i'd like to make myself believe that planet earth turns slowly
it's hard to say that i'd rather stay awake when i'm asleep
'cause everything is never as it seems
'cause i'd get a thousand hugs
from ten thousand lightning bugs
as they tried to teach me how to dance
a foxtrot above my head
a sock hop beneath my bed
a disco ball is just hanging by a thread (thread, thread)
i'd like to make myself believe that planet earth turns slowly
it's hard to say that i'd rather stay awake when i'm asleep
'cause everything is never as it seems (when i fall asleep)
leave my door open just a crack
please take me away from here
'cause i feel like such an insomniac
please take me away from here
why do i tire of counting sheep?
please take me away from here
when i'm far too tired to fall asleep
to ten mi

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [40]:
letters = string.ascii_lowercase + ' ' + '\''
#"".join([i for i in text if i in letters)

In [25]:
correct_text = minimal_token(text, decoder.labels)[1]

speech_file = '/content/vocals.wav'
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

graphemes = decoder.labels
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
text_from_speech = np.array(res)[0]

TypeError: ignored

In [ ]:
text_from_speech.shape

In [ ]:
#forced_alignment_recursive(correct_text, text_from_speech, len(correct_text) - 1, time_values - 1)
#forced_alignment_recursive(correct_text, text_from_speech, 1, 1)
subproblems

In [ ]:
result = forced_alignment(correct_text, text_from_speech)
result = [decoder.labels[correct_text[i]] for i in result[1]]
print(result)

In [ ]:
subproblems

In [ ]:
test = np.array(subproblems)

In [ ]:
test.shape

In [ ]:
test[10][10]

In [ ]:
text_from_speech